In [66]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [67]:
df = pd.read_csv('../Data/heart.csv')

In [68]:
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,59,1,1,140,221,0,1,164,1,0.0,2,0,2,1
1021,60,1,0,125,258,0,0,141,1,2.8,1,1,3,0
1022,47,1,0,110,275,0,0,118,1,1.0,1,1,2,0
1023,50,0,0,110,254,0,0,159,0,0.0,2,0,2,1


In [69]:
X = df.drop('target', axis=1).values
y = df['target'].values

In [70]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [71]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [72]:
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

In [73]:
def build_model(dropout1=0.0, dropout2=0.0):
    layers = [
        nn.Linear(X_train.shape[1], 32),
        nn.ReLU(),
    ]

    if dropout1 > 0.0:
        layers.append(nn.Dropout(dropout1))

    layers.extend([
        nn.Linear(32, 16),
        nn.ReLU(),
    ])

    if dropout2 > 0.0:
        layers.append(nn.Dropout(dropout2))

    layers.extend([
        nn.Linear(16, 2)
    ])

    return nn.Sequential(*layers)

In [74]:
def train_with_dropout(dropout1=0.5, dropout2=0.3, epochs=100, lr=0.01):
    model = build_model(dropout1=dropout1, dropout2=dropout2)
    optimizer = optim.SGD(model.parameters(), lr=lr)
    loss_fun = nn.CrossEntropyLoss()

    history = {
        'test_loss':[],
        'train_loss':[],
        'test_acc':[],
        'train_acc':[]
    }

    for epoch in range(epochs):
        model.train() # 
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = loss_fun(outputs, y_train)

        # train_acc
        _, train_predictions = torch.max(outputs, 1)
        train_acc = (train_predictions == y_train).sum().item() / len(y_train)

        loss.backward()
        optimizer.step()

        with torch.no_grad():
            test_outputs = model(X_test)
            test_loss = loss_fun(test_outputs, y_test)
            _, predicted = torch.max(test_outputs, 1) # _ = abaikan
            acc = (predicted == y_test).sum() / len(y_test) # test accuracy

        history['test_acc'].append(acc)
        history['train_acc'].append(train_acc)
        history['train_loss'].append(loss.item())
        history['test_loss'].append(test_loss)

    return history


In [75]:
def train_with_l1(l1_lambda=0.001, epochs=100, lr=0.01):
    model = build_model()
    optimizer = optim.SGD(model.parameters(), lr=lr)
    loss_fun = nn.CrossEntropyLoss()

    history = {
        'test_loss':[],
        'train_loss':[],
        'test_acc':[],
        'train_acc':[]
    }

    for epoch in range(epochs):
        model.train() # 
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = loss_fun(outputs, y_train)

        l1_regu = sum(p.abs().sum() for p in model.parameters())
        loss = loss + l1_lambda * l1_regu

        # train_acc
        _, train_predictions = torch.max(outputs, 1)
        train_acc = (train_predictions == y_train).sum().item() / len(y_train)

        loss.backward()
        optimizer.step()

        with torch.no_grad():
            test_outputs = model(X_test)
            test_loss = loss_fun(test_outputs, y_test)
            _, predicted = torch.max(test_outputs, 1) # _ = abaikan
            acc = (predicted == y_test).sum() / len(y_test) # test accuracy

        history['test_acc'].append(acc)
        history['train_acc'].append(train_acc)
        history['train_loss'].append(loss.item())
        history['test_loss'].append(test_loss)

    return history


In [ ]:
def train_with_l2(l2_lambda=0.01, epochs=100, lr=0.01):
    model = build_model()
    optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=l2_lambda)
    loss_fun = nn.CrossEntropyLoss()

    history = {
        'test_loss':[],
        'train_loss':[],
        'test_acc':[],
        'train_acc':[]
    }

    for epoch in range(epochs):
        model.train() # 
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = loss_fun(outputs, y_train)

        # train_acc
        _, train_predictions = torch.max(outputs, 1)
        train_acc = (train_predictions == y_train).sum().item() / len(y_train)

        loss.backward()
        optimizer.step()

        with torch.no_grad():
            test_outputs = model(X_test)
            test_loss = loss_fun(test_outputs, y_test)
            _, predicted = torch.max(test_outputs, 1) # _ = abaikan
            acc = (predicted == y_test).sum() / len(y_test) # test accuracy

        history['test_acc'].append(acc)
        history['train_acc'].append(train_acc)
        history['train_loss'].append(loss.item())
        history['test_loss'].append(test_loss)

    return history


In [79]:
history_dropout = train_with_dropout(epochs=100, lr=0.1)

In [82]:
history_dropout['test_acc'][-1]

tensor(0.8049)

In [30]:
# 1. Load dataset heart.csv
df = pd.read_csv("../Data/heart.csv")

# Asumsi kolom terakhir adalah target
X = df.drop("target", axis=1).values
y = df["target"].values

In [31]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [32]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [33]:
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)


In [34]:
def build_model(dropout1=0.0, dropout2=0.0):
    layers = [
        nn.Linear(X_train.shape[1], 32),
        nn.ReLU(),
    ]

    if dropout1 > 0:
        layers.append(nn.Dropout(dropout1))

    layers.extend([
        nn.Linear(32, 16),
        nn.ReLU(),
    ])

    if dropout2 > 0:
        layers.append(nn.Dropout(dropout2))
        
    layers.extend([
        nn.Linear(16, 2)
    ])
    return nn.Sequential(*layers)

In [35]:
def train_with_dropout(dropout1=0.5, dropout2=0.3, epochs=50):
    print(f"\n=== Training with Dropout={dropout1} ===")
    model = build_model(dropout1=dropout1, dropout2=dropout2)
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()

        if (epoch+1) % 10 == 0:
            with torch.no_grad():
                preds = model(X_test)
                _, predicted = torch.max(preds, 1)
                acc = (predicted == y_test).sum().item() / len(y_test)
            print(f"Epoch {epoch+1}: Loss={loss.item():.4f}, Acc={acc:.4f}")

In [36]:
def train_with_l2(l2_lambda=0.01, epochs=50):
    print(f"\n=== Training with L2 (lambda={l2_lambda}) ===")
    model = build_model(dropout1=0.0, dropout2=0.0)
    optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=l2_lambda)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()

        if (epoch+1) % 10 == 0:
            with torch.no_grad():
                preds = model(X_test)
                _, predicted = torch.max(preds, 1)
                acc = (predicted == y_test).sum().item() / len(y_test)
            print(f"Epoch {epoch+1}: Loss={loss.item():.4f}, Acc={acc:.4f}")

In [37]:
def train_with_l1(l1_lambda=0.001, epochs=50):
    print(f"\n=== Training with L1 (lambda={l1_lambda}) ===")
    model = build_model(dropout1=0.0, dropout2=0.0)
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)

        # Tambahkan L1 penalty
        l1_norm = sum(p.abs().sum() for p in model.parameters())
        loss = loss + l1_lambda * l1_norm

        loss.backward()
        optimizer.step()

        if (epoch+1) % 10 == 0:
            with torch.no_grad():
                preds = model(X_test)
                _, predicted = torch.max(preds, 1)
                acc = (predicted == y_test).sum().item() / len(y_test)
            print(f"Epoch {epoch+1}: Loss={loss.item():.4f}, Acc={acc:.4f}")

In [38]:
# 5. Eksperimen
train_with_dropout(dropout1=0.5, dropout2=0.3, epochs=50)
train_with_l2(l2_lambda=0.01, epochs=50)
train_with_l1(l1_lambda=0.001, epochs=50)


=== Training with Dropout=0.5 ===
Epoch 10: Loss=0.5172, Acc=0.7951
Epoch 20: Loss=0.3918, Acc=0.8146
Epoch 30: Loss=0.3547, Acc=0.8439
Epoch 40: Loss=0.3069, Acc=0.8341
Epoch 50: Loss=0.3072, Acc=0.8488

=== Training with L2 (lambda=0.01) ===
Epoch 10: Loss=0.3772, Acc=0.8195
Epoch 20: Loss=0.3143, Acc=0.8488
Epoch 30: Loss=0.2758, Acc=0.8683
Epoch 40: Loss=0.2243, Acc=0.8878
Epoch 50: Loss=0.1719, Acc=0.9268

=== Training with L1 (lambda=0.001) ===
Epoch 10: Loss=0.4979, Acc=0.8049
Epoch 20: Loss=0.4271, Acc=0.8488
Epoch 30: Loss=0.3772, Acc=0.8683
Epoch 40: Loss=0.3389, Acc=0.8878
Epoch 50: Loss=0.3009, Acc=0.9024
